In [25]:
import pandas as pd

회원정보07 = pd.read_parquet(r"..\1.회원정보\201807_train_회원정보.parquet")
회원정보08 = pd.read_parquet(r"..\1.회원정보\201808_train_회원정보.parquet")
회원정보09 = pd.read_parquet(r"..\1.회원정보\201809_train_회원정보.parquet")
회원정보10 = pd.read_parquet(r"..\1.회원정보\201810_train_회원정보.parquet")
회원정보11 = pd.read_parquet(r"..\1.회원정보\201811_train_회원정보.parquet")
회원정보12 = pd.read_parquet(r"..\1.회원정보\201812_train_회원정보.parquet")

마케팅정보07 = pd.read_parquet(r"..\7.마케팅정보\201807_train_마케팅정보.parquet")
마케팅정보08 = pd.read_parquet(r"..\7.마케팅정보\201808_train_마케팅정보.parquet")
마케팅정보09 = pd.read_parquet(r"..\7.마케팅정보\201809_train_마케팅정보.parquet")
마케팅정보10 = pd.read_parquet(r"..\7.마케팅정보\201810_train_마케팅정보.parquet")
마케팅정보11 = pd.read_parquet(r"..\7.마케팅정보\201811_train_마케팅정보.parquet")
마케팅정보12 = pd.read_parquet(r"..\7.마케팅정보\201812_train_마케팅정보.parquet")

성과정보07 = pd.read_parquet(r"..\8.성과정보\201807_train_성과정보.parquet")
성과정보08 = pd.read_parquet(r"..\8.성과정보\201808_train_성과정보.parquet")
성과정보09 = pd.read_parquet(r"..\8.성과정보\201809_train_성과정보.parquet")
성과정보10 = pd.read_parquet(r"..\8.성과정보\201810_train_성과정보.parquet")
성과정보11 = pd.read_parquet(r"..\8.성과정보\201811_train_성과정보.parquet")
성과정보12 = pd.read_parquet(r"..\8.성과정보\201812_train_성과정보.parquet")


In [35]:
import pandas as pd

# 회원정보 결합
회원정보 = pd.concat([회원정보07, 회원정보08, 회원정보09, 회원정보10, 회원정보11, 회원정보12], ignore_index=True)
회원정보 = 회원정보[['기준년월','ID','Segment']]

# 마케팅정보 결합
마케팅정보 = pd.concat([마케팅정보07, 마케팅정보08, 마케팅정보09, 마케팅정보10, 마케팅정보11, 마케팅정보12], ignore_index=True)

# 성과정보 결합
성과정보 = pd.concat([성과정보07, 성과정보08, 성과정보09, 성과정보10, 성과정보11, 성과정보12], ignore_index=True)


In [71]:
# 회원정보와 마케팅정보 병합
df = pd.merge(회원정보, 마케팅정보, on=['기준년월', 'ID'], how='left')

# 위 결과와 성과정보 병합
final_df = pd.merge(df, 성과정보, on=['기준년월', 'ID'], how='left')


In [72]:
# 확인하고 싶은 컬럼 리스트
date_columns = [
    '기준년월',
    '입회일자_신용',
    '최종유효년월_신용_이용가능',
    '최종유효년월_신용_이용',
    '최종카드발급일자',
    'RV신청일자',
    '최종이용일자_기본',
    '최종이용일자_신판',
    '최종이용일자_CA',
    '최종이용일자_카드론',
    '최종이용일자_체크',
    '최종이용일자_일시불',
    '최종이용일자_할부',
    '최종카드론_대출일자'
]

# 실제 존재하는 컬럼만 추출
existing_columns = [col for col in date_columns if col in final_df.columns]

# 결과 출력
print("final_df에 존재하는 날짜 관련 컬럼:")
print(existing_columns)

final_df에 존재하는 날짜 관련 컬럼:
['기준년월']


In [73]:
# 기준년월: datetime 변환
final_df['기준년월'] = pd.to_datetime(
    final_df['기준년월'].astype(str).where(final_df['기준년월'].astype(str).str.match(r'^\d{6}$')),
    format='%Y%m',
    errors='coerce'
)

In [74]:
# 데이터 타입이 object이거나 category인 컬럼 추출
categorical_columns = final_df.select_dtypes(include=['object', 'category']).columns.tolist()

print("범주형 변수 목록:")
print(categorical_columns)

범주형 변수 목록:
['ID', 'Segment', '캠페인접촉건수_R12M', '캠페인접촉일수_R12M']


In [75]:
# 범주형 변수 추출
categorical_columns = final_df.select_dtypes(include=['object', 'category']).columns

# 유일한 값이 하나만 존재하는 컬럼 찾기
single_value_columns = [col for col in categorical_columns if final_df[col].nunique(dropna=False) == 1]

print("값이 하나뿐인 범주형 변수:")
print(single_value_columns)

값이 하나뿐인 범주형 변수:
[]


In [76]:
len(categorical_columns)

4

In [77]:
# 결측치가 있는 칼럼과 결측치 수
null_columns = final_df.isnull().sum()
null_columns = null_columns[null_columns > 0].sort_values(ascending=False)

# 보기 좋게 DataFrame으로 출력
null_df = null_columns.reset_index()
null_df.columns = ['Column', 'MissingValues']

print(null_df)

      Column  MissingValues
0  혜택수혜율_B0M         555522
1  혜택수혜율_R3M         488746


In [78]:
final_df[['혜택수혜율_B0M', '혜택수혜율_R3M']] = final_df[['혜택수혜율_B0M', '혜택수혜율_R3M']].fillna(0)

In [79]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# 날짜 컬럼 리스트 (입회일자 및 최종이용일자 관련)
date_cols = [
    '입회일자_신용', '최종이용일자_기본', '최종이용일자_신판', '최종이용일자_CA',
    '최종이용일자_카드론', '최종이용일자_체크', '최종이용일자_일시불', '최종이용일자_할부'
]

# final_df에 존재하는 컬럼만 선택
existing_date_cols = [col for col in date_cols if col in final_df.columns]

# 각 컬럼에 대해 이상치 처리 및 라벨 인코딩 수행
for col in existing_date_cols:
    # 문자열로 변환하고 이상치 '10101'을 '0'으로 대체
    final_df[col] = final_df[col].astype(str).replace('10101', '0')
    # 라벨 인코딩
    le = LabelEncoder()
    final_df[col] = le.fit_transform(final_df[col])


In [80]:
# 변환 대상 컬럼 리스트
category_cols = ['최종카드론_금융상환방식코드',
    '남녀구분코드', '회원여부_이용가능', '회원여부_이용가능_CA',
    '회원여부_이용가능_카드론', '소지여부_신용', '회원여부_연체',
    '이용거절여부_카드론', '동의여부_한도증액안내', '수신거부여부_TM',
    '수신거부여부_DM', '수신거부여부_메일', '수신거부여부_SMS',
    '마케팅동의여부', '보유여부_해외겸용_본인', '이용가능여부_해외겸용_본인',
    '이용여부_3M_해외겸용_본인', '보유여부_해외겸용_신용_본인',
    '이용가능여부_해외겸용_신용_본인', '이용여부_3M_해외겸용_신용_본인',
    '일시불ONLY전환가능여부', '연체감액여부_R3M', '시장단기연체여부_R6M',
    '시장단기연체여부_R3M', '청구서발송여부_B0', '청구서발송여부_R3M',
    '청구서발송여부_R6M'
]
# 실제 존재하는 컬럼만 선택
existing_cols = [col for col in category_cols if col in final_df.columns]
# 선택된 컬럼들을 category 타입으로 변환
final_df[existing_cols] = final_df[existing_cols].astype('category')

In [81]:
existing_cols

[]

In [82]:
print(categorical_columns)

Index(['ID', 'Segment', '캠페인접촉건수_R12M', '캠페인접촉일수_R12M'], dtype='object')


In [83]:
from sklearn.preprocessing import LabelEncoder

# 예시: 'Segment' 열에 라벨 인코딩 적용
le = LabelEncoder()
final_df['Segment'] = le.fit_transform(final_df['Segment'])
final_df['캠페인접촉건수_R12M'] = le.fit_transform(final_df['캠페인접촉건수_R12M'])
final_df['캠페인접촉일수_R12M'] = le.fit_transform(final_df['캠페인접촉일수_R12M'])


In [84]:
final_df

,기준년월,ID,Segment,컨택건수_카드론_TM_B0M,컨택건수_리볼빙_TM_B0M,컨택건수_CA_TM_B0M,컨택건수_이용유도_TM_B0M,컨택건수_신용발급_TM_B0M,컨택건수_부대서비스_TM_B0M,컨택건수_포인트소진_TM_B0M,...,변동률_RV일시불평잔,변동률_할부평잔,변동률_CA평잔,변동률_RVCA평잔,변동률_카드론평잔,변동률_잔액_B1M,변동률_잔액_일시불_B1M,변동률_잔액_CA_B1M,혜택수혜율_R3M,혜택수혜율_B0M
0,2018-07-01,TRAIN_000000,3,0,0,0,0,0,0,0,...,0.999998,1.042805,0.999700,0.999998,0.999998,0.261886,0.270752,0.000000,1.044401,1.280543
1,2018-07-01,TRAIN_000001,4,3,0,0,0,0,0,0,...,1.092698,0.905663,0.999998,0.999998,0.999998,-0.563388,-0.670348,0.000000,0.000000,0.000000
2,2018-07-01,TRAIN_000002,2,0,0,0,0,0,0,0,...,1.006124,1.993590,0.852567,0.999998,0.999998,-0.046516,0.058114,-0.014191,0.524159,1.208420
3,2018-07-01,TRAIN_000003,3,0,0,0,0,0,0,0,...,0.999998,1.050646,0.999877,0.999998,0.999998,0.023821,0.258943,0.000000,0.880925,1.657124
4,2018-07-01,TRAIN_000004,4,0,0,0,0,0,0,0,...,0.999998,0.999998,0.999998,0.999998,0.999998,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2399995,2018-12-01,TRAIN_399995,4,0,0,0,0,0,0,0,...,0.999998,0.999998,0.999998,0.999998,0.999998,0.000000,0.000000,0.000000,0.000000,0.000000
2399996,2018-12-01,TRAIN_399996,3,0,0,0,0,0,0,0,...,0.999998,0.999998,0.999998,0.999998,0.921733,-0.203251,-0.159143,0.000000,1.377071,2.533815
2399997,2018-12-01,TRAIN_399997,2,0,0,0,0,0,0,0,...,0.999998,0.345027,0.999998,0.999998,0.999998,0.027319,0.126581,0.000000,0.000000,0.000000
2399998,2018-12-01,TRAIN_399998,4,0,0,0,0,0,0,0,...,0.999998,0.999998,0.999998,0.999998,0.999998,0.000000,0.000000,0.000000,0.000000,0.000000


In [85]:
# 제외할 칼럼 목록
exclude_cols = ['ID', '기준년월']

# 상관계수 계산 (Segment 기준)
correlations = final_df.drop(columns=exclude_cols).corr(numeric_only=True)['Segment'].drop('Segment')

# 상관계수 0.1 이상인 피처 추출
high_corr_features = correlations[correlations.abs() >= 0.1]

# 결과 출력
print("Segment와 상관계수(|r| ≥ 0.1)인 피처 수:", len(high_corr_features))
print("\n해당 피처들:\n", high_corr_features.sort_values(ascending=False))

Segment와 상관계수(|r| ≥ 0.1)인 피처 수: 13

해당 피처들:
 컨택건수_이용유도_TM_R6M      0.149037
변동률_RVCA평잔           -0.104127
변동률_RV일시불평잔          -0.104848
컨택건수_이용유도_EM_B0M     -0.127584
컨택건수_이용유도_EM_R6M     -0.133304
잔액_신판평균한도소진율_r6m     -0.181995
잔액_신판평균한도소진율_r3m     -0.189758
잔액_신판최대한도소진율_r3m     -0.194820
잔액_신판최대한도소진율_r6m     -0.200087
잔액_신판ca평균한도소진율_r6m   -0.252525
잔액_신판ca평균한도소진율_r3m   -0.258636
잔액_신판ca최대한도소진율_r3m   -0.263688
잔액_신판ca최대한도소진율_r6m   -0.268486
Name: Segment, dtype: float64


In [91]:
selected_cols = ['ID', '기준년월', 'Segment'] + list(high_corr_features.index)
filtered_df = final_df[selected_cols]
filtered_df.to_csv('0.1이상 상관계수(78).csv', index=False, encoding='utf-8-sig')